In [209]:
# time_constraint = 15000
# path_raw = "ID: 0->ID: 3->ID: 8->ID: 10->ID: 16->ID: 9->ID: 6->ID: 2->ID: 1->ID: 4->ID: 17->ID: 26->ID: 35->ID: 37->ID: 34->ID: 28->ID: 12->ID: 22->ID: 25->ID: 19->ID: 18->ID: 20->ID: 23->ID: 36->ID: 21->ID: 13->ID: 29->ID: 15->ID: 14->ID: 41->ID: 42->ID: 38->ID: 5->ID: 11->ID: 7->ID: 0->"
# legend_placement = [52.532883, 13.339717]

time_constraint = 30000
path_raw = "ID: 0->ID: 3->ID: 8->ID: 10->ID: 16->ID: 9->ID: 6->ID: 2->ID: 1->ID: 4->ID: 17->ID: 26->ID: 35->ID: 37->ID: 34->ID: 28->ID: 12->ID: 22->ID: 25->ID: 19->ID: 18->ID: 20->ID: 23->ID: 36->ID: 21->ID: 13->ID: 29->ID: 15->ID: 14->ID: 41->ID: 42->ID: 38->ID: 5->ID: 11->ID: 7->ID: 30->ID: 24->ID: 32->ID: 33->ID: 27->ID: 31->ID: 40->ID: 46->ID: 45->ID: 48->ID: 56->ID: 52->ID: 62->ID: 58->ID: 68->ID: 65->ID: 73->ID: 50->ID: 51->ID: 94->ID: 0->"
legend_placement = [52.706653, 12.874888]

In [210]:
import locale
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [211]:
extra_html = ""

In [212]:
from context import openroute

openroute.api.go_offline()
local_ready = openroute.check_status()
print("[{}] localhost".format(' OK ' if local_ready else 'FAIL'))

openroute.api.go_online()
azure_ready = openroute.check_status()
print("[{}] azure".format(' OK ' if azure_ready else 'FAIL'))

openroute.api.toggle_online(not local_ready)

print('Using: {}'.format('azure' if openroute.api.is_online() else 'localhost'))

[FAIL] localhost
[ OK ] azure
Using: azure


In [213]:
import json

with open("data/matrix500.json", 'r') as infile:
    data = json.load(infile)

geocaches = [[lat, lon] for lon, lat in data['metadata']['query']['locations']]

In [214]:
path = [int(val) for val in list(filter(None, path_raw.replace('ID: ', '').split('->')))]

In [215]:
time_constraint = 0
legend_placement = [52.538649, 13.326358]

ant_count = 10000
iterations = 74
runtime = 214369
cost = 25416.929999999997
path = [0, 3, 10, 8, 16, 41, 42, 38, 9, 14, 15, 29, 13, 36, 43, 39, 23, 20, 18, 19, 25, 22, 34, 28, 17, 12, 31, 27, 35, 37, 26, 32, 40, 48, 46, 45, 24, 33, 30, 11, 5, 7, 49, 47, 44, 21, 6, 2, 1, 4]

extra_html = f"""
    <tr>
        <td>Ants:</td><td align=right>{ant_count:n}</td>
    </tr>
    <tr>
        <td>Runtime:</td><td align=right>{str(datetime.timedelta(seconds=round(runtime/1000)))}</td>
    </tr>
    <tr>
        <td>Iterations:</td><td align=right>{iterations:n}</td>
    </tr>
"""

In [216]:
print(path)

[0, 3, 10, 8, 16, 41, 42, 38, 9, 14, 15, 29, 13, 36, 43, 39, 23, 20, 18, 19, 25, 22, 34, 28, 17, 12, 31, 27, 35, 37, 26, 32, 40, 48, 46, 45, 24, 33, 30, 11, 5, 7, 49, 47, 44, 21, 6, 2, 1, 4]


In [217]:
import folium
from algo_results import create_marker, create_circle_marker, fit_locations, style_function, create_legend

In [218]:
import math
max_geocaches = int(math.ceil((max(path) + 1) / 10.0)) * 10

In [219]:
map = folium.Map(tiles=None, location=geocaches[path[0]], zoom_start=14, zoom_control=False)
folium.TileLayer(tiles='Stamen Toner', opacity=0.15).add_to(map)
pass

In [220]:
from tqdm import tqdm

optimal_locations = [[geocaches[id][1], geocaches[id][0]] for id in path]

step = 10
max_steps = math.ceil(len(optimal_locations) / step)

In [221]:
import colorsys
import seaborn as sns

color_palette = ['#%02x%02x%02x' % (int(color[0]*255), int(color[1]*255), int(color[2]*255)) for color in sns.color_palette("hls", max_steps)]

color_palette_html = f"""
    <table class="color-palette"><tr>{''.join([f'<td style="color: {color};"><div>&nbsp;</div></td>' for color in color_palette])}</tr></table>
"""

color_palette

['#db5e56', '#b8db56', '#56db93', '#5683db', '#c856db']

In [222]:
optimal_routes = []

for index in tqdm(range(max_steps)):
    start = index * step
    request = {
        'coordinates': optimal_locations[start:start+step+1],
        'geometry': True,
        'instructions': False
    }
    optimal_route = openroute.request_directions(request, format_out='/geojson')

    if optimal_route:
        folium.features.GeoJson(data=optimal_route,
                                name='Optimal route',
                                style_function=style_function(color_palette[index]),
                                overlay=True).add_to(map)
        optimal_routes.append(optimal_route)

fit_locations(map, geocaches[:max_geocaches])

100%|██████████| 5/5 [00:02<00:00,  2.31it/s]


In [223]:
use_penultimate = path[0] == path[-1]
start_node = path[0]
last_node = path[-2] if use_penultimate else path[-1]
visited_nodes = len(path) - 1 if use_penultimate else len(path)

for index, coords in enumerate(geocaches[:max_geocaches]):
    color = '#787c99'
    if index == start_node: # start
        color = '#41a6b5'
    elif index == last_node: # last node before start
        color = '#9d7cd8'
    elif index in path: # other visited
        color = '#3c7da2'
    create_marker(path, coords, index, color, radius=8).add_to(map)

In [224]:
import datetime

def sum_duration(routes: list):
    sum = 0.0
    for route in routes:
        sum += route['features'][0]['properties']['summary']['duration']
    return sum

optimal_duration = cost or sum_duration(optimal_routes)
time_str = str(datetime.timedelta(seconds=round(optimal_duration)))

time_constraint_html = ''
if time_constraint:
    time_constraint_html = f"""
        <tr>
            <td>Time constraint:</td>
            <td align=right>{str(datetime.timedelta(seconds=time_constraint))}</td>
        </tr>
    """

legend = create_legend(legend_placement, html=f"""
    {color_palette_html}
    <table width=100%>
        {extra_html}
        <tr>
            <td>Visited nodes:</td>
            <td align=right>{visited_nodes}</td>
        </tr>
        <tr>
            <td>Total duration:</td>
            <td align=right>{time_str}</td>
        </tr>
        {time_constraint_html}
    </table>
    <div class="marker-description">
        <span class="start">Start node</span>
        <span class="last">{"Penultimate" if use_penultimate else "Last"} node</span>
    </div>
""").add_to(map)

(optimal_duration, time_str)

(25416.929999999997, '7:03:37')

In [225]:
map